In [1]:

import pandas as pd
import polars as pl
from utils import *

main_folder = 'snapshots'
# Stations master file
#URL: https://opendata-ajuntament.barcelona.cat/data/es/dataset/informacio-estacions-bicing
stations_master_file = main_folder+ '/2024_05_Maig_BicingNou_INFORMACIO.csv'

# Polars
#stations_master = pl.read_csv(stations_master_file, ignore_errors = True)

# Pandas
stations_master = pd.read_csv(stations_master_file).sort_values(by='station_id')
stations_master['last_updated'] = pd.to_datetime(stations_master['last_updated'])
stations_master = stations_master.sort_values('last_updated').groupby('station_id').last()


#stations_master.sort_values(by='altitude', ascending=False, inplace=True)
stations_master#.head(5)

/var/folders/md/jgs9sl817gxfmk03qhzt6bk80000gq/T/ipykernel_47875/1184890306.py:14: DtypeWarning: Columns (9,12) have mixed types. Specify dtype option on import or set low_memory=False.
  stations_master = pd.read_csv(stations_master_file).sort_values(by='station_id')


,name,physical_configuration,lat,lon,altitude,address,post_code,capacity,is_charging_station,short_name,nearby_distance,x_ride_code_support,rental_uris,cross_street,last_updated,ttl,V1
station_id,,,,,,,,,,,,,,,,,
1.0,"GRAN VIA CORTS CATALANES, 760",ELECTRICBIKESTATION,41.397978,2.180107,16.0,"GRAN VIA CORTS CATALANES, 760",8013.0,46.0,True,1.0,1000.0,True,NaN,None,1970-01-01 00:00:01.717193703,0.0,NaN
2.0,"C/ ROGER DE FLOR, 126",ELECTRICBIKESTATION,41.395488,2.177198,17.0,"C/ ROGER DE FLOR, 126",8013.0,29.0,True,2.0,1000.0,True,NaN,None,1970-01-01 00:00:01.717193703,0.0,NaN
3.0,"C/ NÀPOLS, 82",ELECTRICBIKESTATION,41.394156,2.181331,11.0,"C/ NÀPOLS, 82",8013.0,27.0,True,3.0,1000.0,True,NaN,None,1970-01-01 00:00:01.717193703,0.0,NaN
4.0,"C/ RIBES, 13",ELECTRICBIKESTATION,41.393317,2.181248,8.0,"C/ RIBES, 13",8013.0,21.0,True,4.0,1000.0,True,NaN,None,1970-01-01 00:00:01.717193703,0.0,NaN
5.0,"PG. LLUIS COMPANYS, 11 (ARC TRIOMF)",ELECTRICBIKESTATION,41.391103,2.180176,7.0,"PG. LLUIS COMPANYS, 11 (ARC TRIOMF)",8018.0,39.0,True,5.0,1000.0,True,NaN,None,1970-01-01 00:00:01.717193703,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
516.0,"C/ SANT ADRIÀ, 88",ELECTRICBIKESTATION,41.435460,2.200157,15.0,"C/ SANT ADRIÀ, 88",8030.0,21.0,True,516.0,1000.0,True,NaN,None,1970-01-01 00:00:01.717193703,0.0,NaN
517.0,"AV. RASOS DE PEGUERA, 10",ELECTRICBIKESTATION,41.462095,2.178959,44.0,"AV. RASOS DE PEGUERA, 10",8033.0,20.0,True,517.0,1000.0,True,NaN,None,1970-01-01 00:00:01.717193703,0.0,NaN
518.0,"C/ LLOBREGÓS, 115",ELECTRICBIKESTATION,41.424689,2.157049,112.0,"C/ LLOBREGÓS, 115",8032.0,27.0,True,518.0,1000.0,True,NaN,None,1970-01-01 00:00:01.717193703,0.0,NaN


In [2]:
# Bike stations by post code
import folium
import random

available_colors = [
    'lightgreen', 'pink', 'beige', 'gray', 'green', 'cadetblue', 'blue', 
    'black', 'lightred', 'darkblue', 'lightblue', 'darkred', 'lightgray', 
    'orange', 'darkgreen', 'red', 'white', 'darkpurple', 'purple'
]

def random_color(postcode, available_colors=available_colors):
    random.seed(postcode)
    color = random.choice(available_colors)
    random.seed(None)  
    return color

# Create a map object with Barcelona as the center
center_coords = [41.41, 2.2]
m_postcode = folium.Map(location=center_coords, zoom_start=12)


for index, row in stations_master.iterrows():
    color = random_color(row['post_code'], available_colors)
    if color == 'white':
        icon_color = 'black'
    else: 
        icon_color = 'white'

    folium.Marker(location=[row['lat'], row['lon']], popup= (str(int(row['short_name']))+ ': ' + row['name'] + ' PC: '+ str(int(row['post_code'])) + ' Capacity: '+ str(int(row['capacity'])) + ' Altitude: ' + str(int(row['altitude'])) )
                  ,  icon=folium.Icon(icon='bicycle', color= color, icon_color= icon_color, prefix='fa')).add_to(m_postcode)

In [3]:
districts_geojson = 'snapshots/districtes.geojson'
# Create a map object with Barcelona as the center
center_coords = [41.41, 2.2]
m_district = folium.Map(location=center_coords, zoom_start=12)

stations_master2 = add_districts(stations_master)
for index, row in stations_master2.iterrows():
    color = random_color(row['district'], available_colors)
    if color == 'white':
        icon_color = 'black'
    else: 
        icon_color = 'white'

    #folium.GeoJson(districts_geojson).add_to(m_district)
    folium.Marker(location=[row['lat'], row['lon']], popup= (str(int(row['short_name']))+ ': ' + row['name'] + ' PC: '+ str(int(row['post_code'])) + ' Capacity: '+ str(int(row['capacity'])) + ' Altitude: ' + str(int(row['altitude'])) + ' District: ' + str(row['district']) )
                  ,  icon=folium.Icon(icon='bicycle', color= color, icon_color= icon_color, prefix='fa')).add_to(m_district)

In [4]:
suburb_geojson = 'snapshots/barris.geojson'
# Create a map object with Barcelona as the center
center_coords = [41.41, 2.2]
m_suburb = folium.Map(location=center_coords, zoom_start=12)

stations_master2 = add_suburb(stations_master)
for index, row in stations_master2.iterrows():
    color = random_color(row['suburb'], available_colors)
    if color == 'white':
        icon_color = 'black'
    else: 
        icon_color = 'white'

    #folium.GeoJson(suburb_geojson).add_to(m_suburb)
    folium.Marker(location=[row['lat'], row['lon']], popup= (str(int(row['short_name']))+ ': ' + row['name'] + ' PC: '+ str(int(row['post_code'])) + ' Capacity: '+ str(int(row['capacity'])) + ' Altitude: ' + str(int(row['altitude'])) + ' District: ' + str(row['district']) + ' Suburb: ' + str(row['suburb']) )
                  ,  icon=folium.Icon(icon='bicycle', color= color, icon_color= icon_color, prefix='fa')).add_to(m_suburb)

In [5]:
# Bike stations by altitude
def altitude_to_color(altitude, max_altitude=166, min_altitude=0, available_colors=available_colors):
    num_colors = len(available_colors)
    range_size = (max_altitude - min_altitude) / num_colors
    color_index = int(altitude // range_size)
    if color_index >= num_colors:
        color_index = num_colors - 1
    return available_colors[color_index]

# Create a map object with Barcelona as the center
center_coords = [41.41, 2.2]
m_altitude = folium.Map(location=center_coords, zoom_start=12)


for index, row in stations_master.iterrows():
    color = altitude_to_color(row['altitude'], max_altitude= stations_master['altitude'].max(), min_altitude = stations_master['altitude'].min(), available_colors=available_colors)
    if color == 'white':
        icon_color = 'black'
    else: 
        icon_color = 'white'

    folium.Marker(location=[row['lat'], row['lon']], popup= (str(int(row['short_name']))+ ': ' + row['name'] + ' PC: '+ str(int(row['post_code'])) + ' Capacity: '+ str(int(row['capacity'])) + ' Altitude: ' + str(int(row['altitude'])) )
                  ,  icon=folium.Icon(icon='bicycle', color= color, icon_color= icon_color, prefix='fa')).add_to(m_altitude)


In [6]:
# Bike stations by capacity

#TODO: Add a color gradient
def capacity_to_color(capacity, min_capacity=0, max_capacity=100, available_colors=available_colors):
    num_colors = len(available_colors)
    range_size = (max_capacity - min_capacity) / num_colors
    if capacity < min_capacity:
        capacity = min_capacity
    elif capacity > max_capacity:
        capacity = max_capacity
    color_index = int((capacity - min_capacity) // range_size)
    if color_index >= num_colors:
        color_index = num_colors - 1
    return available_colors[color_index]

# Create a map object with Barcelona as the center
center_coords = [41.41, 2.2]
m_capacity = folium.Map(location=center_coords, zoom_start=12)


for index, row in stations_master.iterrows():
    color = capacity_to_color(row['capacity'], max_capacity= stations_master['capacity'].max(), min_capacity = stations_master['capacity'].min(), available_colors=available_colors)
    if color == 'white':
        icon_color = 'black'
    else: 
        icon_color = 'white'

    folium.Marker(location=[row['lat'], row['lon']], popup= (str(int(row['short_name']))+ ': ' + row['name'] + ' PC: '+ str(int(row['post_code'])) + ' Capacity: '+ str(int(row['capacity'])) + ' Altitude: ' + str(int(row['altitude'])) )
                  ,  icon=folium.Icon(icon='bicycle', color= color, icon_color= icon_color, prefix='fa')).add_to(m_capacity)


In [7]:
m_district
#m_postcode
#m_altitude
#m_capacity
#m_suburb